In [12]:
import torch
import torchvision.models
import torchvision.transforms as T
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import pandas as pd

In [13]:
model = torch.load("./models/newData-15-5-21.pth")
model.eval()
model.cpu()

Model(
  (conv1): Conv2d(1, 10, kernel_size=(10, 10), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(20, 40, kernel_size=(10, 10), stride=(2, 2))
  (fc1): Linear(in_features=60840, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=1, bias=True)
)

In [14]:
trfs = T.Compose([
    T.ToTensor(),
    T.Resize((200,200)),
    # T.Grayscale()
])

all_datasets = ImageFolder("./data", transform=trfs)
loader = DataLoader(all_datasets, batch_size=2048)

In [15]:
with torch.no_grad():
    final = None
    for batch in loader:
        x, y = batch
        pred = model(x)
        pred = pred.argmax(dim=1)
        if final is None:
            final = pred
        else:
            final = torch.cat((final,pred))

In [16]:
final.shape

torch.Size([12521])

In [17]:
num_correct = (final == torch.Tensor(all_datasets.targets)).sum()

In [18]:
percent_correct = (num_correct / len(all_datasets)).item()*100
f"Correct: {percent_correct}%"

'Correct: 83.26811194419861%'

In [19]:
matrix = torch.zeros((2,2))

In [20]:
for pred,y in zip(final.tolist(),all_datasets.targets):
    matrix[pred,y] += 1

In [21]:
matrix_df = pd.DataFrame(matrix.tolist(), columns=["Nothing","Present"],index=["Nothing","Present"])

In [22]:
matrix_df.to_clipboard()

In [23]:
matrix_df

,Nothing,Present
Nothing,10426.0,2095.0
Present,0.0,0.0
